In [1]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://c1aa5f4167a8:4040
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1630583221812)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@42fb13fa


In [12]:
def getTables(query: String): Seq[String] = {
  val logicalPlan = spark.sessionState.sqlParser.parsePlan(query)
  import org.apache.spark.sql.catalyst.analysis.UnresolvedRelation
  logicalPlan.collect { case r: UnresolvedRelation => r.tableName }
}

val query = "select * from table_1 as a left join table_2 as b on a.id=b.id"

getTables: (query: String)Seq[String]
query: String = select * from table_1 as a left join table_2 as b on a.id=b.id


In [13]:
getTables(query).foreach(println)

table_1
table_2


* [org.apache.spark.sql.execution.SparkSqlParser](https://github.com/apache/spark/blob/master/sql/core/src/main/scala/org/apache/spark/sql/execution/SparkSqlParser.scala)
* [SparkSqlParser](https://jaceklaskowski.github.io/mastering-spark-sql-book/sql/SparkSqlParser/)

In [21]:
case class DeviceIoTData (
  battery_level: Long, 
  c02_level: Long,
  cca2: String, 
  cca3: String, 
  cn: String, 
  device_id: Long,
  device_name: String, 
  humidity: Long, 
  ip: String, 
  latitude: Double,
  lcd: String, 
  longitude: Double, 
  scale:String, 
  temp: Long,
  timestamp: Long
)

case class DeviceTempByCountry(
    temp: Long, device_name: String, device_id: Long, cca3: String
)

defined class DeviceIoTData
defined class DeviceTempByCountry


In [19]:
val ds = spark.read.json("/home/jovyan/work/data/databricks/iot_devices.json").as[DeviceIoTData]
val filterTempDS = ds.filter( { d =>  { d.temp > 30 && d.humidity > 70} } )

ds: org.apache.spark.sql.Dataset[DeviceIoTData] = [battery_level: bigint, c02_level: bigint ... 13 more fields]
filterTempDS: org.apache.spark.sql.Dataset[DeviceIoTData] = [battery_level: bigint, c02_level: bigint ... 13 more fields]


In [24]:
ds.printSchema

root
 |-- battery_level: long (nullable = true)
 |-- c02_level: long (nullable = true)
 |-- cca2: string (nullable = true)
 |-- cca3: string (nullable = true)
 |-- cn: string (nullable = true)
 |-- device_id: long (nullable = true)
 |-- device_name: string (nullable = true)
 |-- humidity: long (nullable = true)
 |-- ip: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- lcd: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- scale: string (nullable = true)
 |-- temp: long (nullable = true)
 |-- timestamp: long (nullable = true)



In [28]:
val dsTemp = ds.filter(d => {d.temp > 25})
  .map(d => (d.temp, d.device_name, d.device_id, d.cca3))
  .toDF("temp", "device_name", "device_id", "cca3")
  .as[DeviceTempByCountry]

dsTemp: org.apache.spark.sql.Dataset[DeviceTempByCountry] = [temp: bigint, device_name: string ... 2 more fields]


In [30]:
dsTemp.printSchema()

root
 |-- temp: long (nullable = false)
 |-- device_name: string (nullable = true)
 |-- device_id: long (nullable = false)
 |-- cca3: string (nullable = true)



In [32]:
dsTemp.createOrReplaceTempView("dstab")
spark.sql("select * from dstab").show()

org.apache.spark.SparkException:  Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 6) (c1aa5f4167a8 executor driver): java.lang.ClassCastException: $iw cannot be cast to $iw